# Data Engineering and Preparation
This notebook is aimed at showing the most important steps of the implemented data preparation. Part of the data preparation is calles as a function. Readers interested in all the specifics steps taken should consult the data_handling.py file in the data_handling folder.

In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from data_handling import rawdf_to_stockdfs, get_features, batch_tensor, df_list_to_series_tensor

## Loading the Data

In [2]:
raw_df = pd.read_parquet('../data/data.parquet.gzip')
raw_df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


## Dealing with NA values and amplifying the data with handcrafted features

In [3]:
# In order to use ffill later
raw_df.sort_values(by=['stock_id', 'time_id'], inplace=True)

# Fill NA values with 0 when possible and using the last valid observation otherwise
raw_df.fillna({
    'far_price': 0,
    'near_price': 0,
    'imbalance_size': raw_df['imbalance_size'].ffill(),
    'reference_price': raw_df['reference_price'].ffill(),
    'matched_size': raw_df['matched_size'].ffill(),
    'bid_price': raw_df['bid_price'].ffill(),
    'ask_price': raw_df['ask_price'].ffill(),
    'wap': raw_df['wap'].ffill()
}, inplace=True)

# Amplifying the Data with Features engineered by hand
amp_df = get_features(raw_df)

# Reordering the columns to ensure Targets is the last column
new_order = [col for col in amp_df.columns if col != 'target'] + ['target']
amp_df = amp_df[new_order]

In [4]:
# Dropping the rows with missing targets
amp_df = amp_df.dropna()
assert amp_df.isnull().sum().sum() == 0

## Splitting the data into training, validation and test split

In [5]:
df_train = amp_df[amp_df['date_id'] < 100]
df_train = df_train.sort_values(by=['stock_id', 'time_id'], inplace=False)

df_validation = amp_df[amp_df['date_id'] > 99]
df_validation = df_validation[df_validation['date_id'] < 120]
df_validation = df_validation.sort_values(by=['stock_id', 'time_id'], inplace=False)

df_test = amp_df[amp_df['date_id'] > 119]
df_test = df_test[df_test['date_id'] < 140]
df_test = df_test.sort_values(by=['stock_id', 'time_id'], inplace=False)


## Scaling the data

In [6]:
# Scaling the Data
# The first four will later be dropped and the target column will be scaled differently
excluded_columns = ['stock_id', 'date_id', 'time_id', 'row_id', 'target']

# Standardize only the columns that are not in the excluded list
feature_scaler = StandardScaler()
target_scaler = MinMaxScaler(feature_range=(-1, 1))

# Scaling the Training Data
df_train[df_train.columns.difference(excluded_columns)] = feature_scaler.fit_transform(df_train[df_train.columns.difference(excluded_columns)])
df_train[['target']] = target_scaler.fit_transform(df_train[['target']])

# Scaling the Validation Data
df_validation[df_validation.columns.difference(excluded_columns)] = feature_scaler.transform(df_validation[df_validation.columns.difference(excluded_columns)])
df_validation[['target']] = target_scaler.transform(df_validation[['target']])

# Scaling the Test Data
df_test[df_test.columns.difference(excluded_columns)] = feature_scaler.transform(df_test[df_test.columns.difference(excluded_columns)])
df_test[['target']] = target_scaler.transform(df_test[['target']]) 

## Splitting the data by stock

In [7]:
dflist_train = rawdf_to_stockdfs(df_train)
dflist_validation = rawdf_to_stockdfs(df_validation)
dflist_test = rawdf_to_stockdfs(df_test)

## Creating Sequences and Batches of Sequences from that

We create batches in order to speed up computations and to stabilize the learning process.

In [8]:
# Parameters
sequence_length = 10
batch_size = 64

# Turning data into sequences
collected_tensor_train = df_list_to_series_tensor(dflist_train, sequence_length, shuffle=True)
collected_tensor_test = df_list_to_series_tensor(dflist_test, sequence_length, shuffle=True)
collected_tensor_validation = df_list_to_series_tensor(dflist_validation, sequence_length, shuffle=True)

# Turning the aggregated sequences into batches
batches_train = batch_tensor(collected_tensor_train, batch_size)
batches_validation = batch_tensor(collected_tensor_test, batch_size)
batches_test = batch_tensor(collected_tensor_validation, batch_size)

In [9]:
import os
if not os.path.exists('processed_data'):
    os.makedirs('processed_data')

torch.save(batches_train, 'processed_data/batches_train.pt')
torch.save(batches_validation, 'processed_data/batches_validation.pt')
torch.save(batches_test, 'processed_data/batches_test.pt')